In [12]:
import matplotlib.pyplot as plt
from pylab import arange
from math import e
from math import pi
from math import sin
from math import cos
from numpy import poly1d
import numpy as np
from sympy import *
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import math
import random
random.seed(1234)

In [13]:

def f(x):
  return  np.exp(x)/(np.exp(x)+1)
  
def cal(begin, end, i):
  by = f(begin)
  ey = f(end)
  I = Ms[i] * ((end - n) ** 3) / 6 + Ms[i + 1] * ((n - begin) ** 3) / 6 + (by - Ms[i] / 6) * (end - n) + (ey - Ms[i + 1] / 6) * (n - begin)
  return I


def ff(x):  # f[x0, x1, ..., xk]
    ans = 0
    for i in range(len(x)):
        temp = 1
        for j in range(len(x)):
            if i != j:
                temp *= (x[i] - x[j])
        ans += f(x[i]) / temp
    return ans


def calM():
    lam = [1] + [1 / 2] * 9
    miu = [1 / 2] * 9 + [1]
    # Y = 1 / (1 + n ** 2)
    # df = diff(Y, n)
    #x = np.array(range(11)) - 5
    x = np.random.normal(0, 1, 100)
    x=  np.array(x[0:11]) 
    # ds = [6 * (ff(x[0:2]) - df.subs(n, x[0]))]
    ds = [6 * (ff(x[0:2]) - 1)]
    for i in range(9):
        ds.append(6 * ff(x[i: i + 3]))
    # ds.append(6 * (df.subs(n, x[10]) - ff(x[-2:])))
    ds.append(6 * (1 - ff(x[-2:])))
    Mat = np.eye(11, 11) * 2
    for i in range(11):
        if i == 0:
            Mat[i][1] = lam[i]
        elif i == 10:
            Mat[i][9] = miu[i - 1]
        else:
            Mat[i][i - 1] = miu[i - 1]
            Mat[i][i + 1] = lam[i]
    ds = np.mat(ds)
    Mat = np.mat(Mat)
    Ms = ds * Mat.I
    return Ms.tolist()[0]


def calnf(x):
    nf = []
    for i in range(len(x) - 1):
        nf.append(cal(x[i], x[i + 1], i))
    return nf


def calf(f, x):
    y = []
    for i in x:
        y.append(f.subs(n, i))
    return y


def nfSub(x, nf):
    tempx = np.array(range(11)) - 5
    dx = []
    for i in range(10):
        labelx = []
        for j in range(len(x)):
            if x[j] >= tempx[i] and x[j] < tempx[i + 1]:
                labelx.append(x[j])
            elif i == 9 and x[j] >= tempx[i] and x[j] <= tempx[i + 1]:
                labelx.append(x[j])
        dx = dx + calf(nf[i], labelx)
    return np.array(dx)


def draw(nf):
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    x = np.random.normal(0, 1, 100)
    x=  np.array(x[0:11]) 
    y = f(x)+np.random.normal(0, 1, 11)
    Ly = nfSub(x, nf)
    plt.plot(x, y, label='Primitive')
    plt.plot(x, Ly, label='Cubic spline interpolation function')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()

    plt.savefig('1.png')
    plt.show()


def lossCal(nf):
    x = np.random.normal(0, 1, 100)
    x=  np.array(x[0:11]) 
    y = f(x)+np.random.normal(0, 1, 11)
    Ly = nfSub(x, nf)
    Ly = np.array(Ly)
    temp = Ly - y
    temp = abs(temp)
    print(temp.mean())


if __name__ == '__main__':
    x = np.random.normal(0, 1, 100)
    x=  np.array(x[0:11]) 
    y = f(x)+np.random.normal(0, 1, 11)

    n, m = symbols('n m')
    init_printing(use_unicode=True)
    Ms = calM()
    nf = calnf(x)
    lossCal(nf)

1.35353619086024


In [15]:

class BS_curve(object):

    def __init__(self,n,p,cp=None,knots=None):
        self.n = n # n+1 control points >>> p0,p1,,,pn
        self.p = p
        if cp:
            self.cp = cp
            self.u = knots
            self.m = knots.shape[0]-1 # m+1 knots >>> u0,u1,,,nm
        else:
            self.cp = None
            self.u = None
            self.m = None

        self.paras = None


    def check(self):
        if self.m == self.n + self.p + 1:
            return 1
        else:
            return 0


    def coeffs(self,uq):
        # n+1 control points >>> p0,p1,,,pn
        # m+1 knots >>> u0,u1,,,nm
        # algorithm is from https://pages.mtu.edu/~shene/COURSES/cs3621/NOTES/spline/B-spline/bspline-curve-coef.html
    
        #N[] holds all intermediate and the final results
        # in fact N is longer than control points,this is just to hold the intermediate value
        # at last, we juest extract a part of N,that is N[0:n+1]
        N = np.zeros(self.m+1,dtype=np.float64) 

        # rule out special cases Important Properties of clamped B-spline curve
        if uq == self.u[0]:
            N[0] = 1.0
            return N[0:self.n+1]
        elif uq == self.u[self.m]:
            N[self.n] = 1.0
            return N[0:self.n+1]

        # now u is between u0 and um
        # first find k uq in span [uk,uk+1)
        check = uq - self.u
        ind = check >=0
        k = np.max(np.nonzero(ind))
       
        # sk >>> multiplicity of u[k]
        sk = np.sum(self.u==self.u[k])

        N[k] = 1.0 # degree 0
        # degree d goes from 1 to p
        for d in range(1,self.p+1):
            r_max = self.m - d - 1 # the maximum subscript value of N in degree d,the minimum is 0
            if k-d >=0:
                if self.u[k+1]-self.u[k-d+1]:
                    N[k-d] = (self.u[k+1]-uq)/(self.u[k+1]-self.u[k-d+1])*N[k-d+1] #right (south-west corner) term only
                else:
                    N[k-d] = (self.u[k+1]-uq)/1*N[k-d+1] #right (south-west corner) term only

            for i in range(k-d+1,(k-1)+1):
                if i>=0 and i<=r_max:
                    Denominator1 = self.u[i+d]-self.u[i]
                    Denominator2 = self.u[i+d+1]-self.u[i+1]
                    # 0/0=0
                    if Denominator1 == 0:
                        Denominator1 = 1
                    if Denominator2 == 0:
                        Denominator2 = 1

                    N[i] = (uq-self.u[i])/(Denominator1)*N[i]+(self.u[i+d+1]-uq)/(Denominator2)*N[i+1]

            if k <= r_max:
                if self.u[k+d]-self.u[k]:
                    N[k] = (uq-self.u[k])/(self.u[k+d]-self.u[k])*N[k]
                else:
                    N[k] = (uq-self.u[k])/1*N[k]

        return N[0:self.n+1]


    def De_Boor(self,uq):
        # Input: a value u
        # Output: the point on the curve, C(u)

        # first find k uq in span [uk,uk+1)
        check = uq - self.u
        ind = check >=0
        k = np.max(np.nonzero(ind))
        
        # inserting uq h times
        if uq in self.u:
            # sk >>> multiplicity of u[k]
            sk = np.sum(self.u==self.u[k])
            h = self.p - sk
        else:
            sk = 0
            h = self.p

        # rule out special cases
        if h == -1:
            if k == self.p:
                return np.array(self.cp[0])
            elif k == self.m:
                return np.array(self.cp[-1])


        # initial values of P(affected control points) >>> Pk-s,0 Pk-s-1,0 ... Pk-p+1,0
        P = self.cp[k-self.p:k-sk+1]
        P = P.copy()
        dis = k-self.p # the index distance between storage loaction and varibale i
        # 1-h
        
        for r in range(1,h+1):
            # k-p >> k-sk
            temp = [] # uesd for Storing variables of the current stage
            for i in range(k-self.p+r,k-sk+1):
                a_ir = (uq-self.u[i])/(self.u[i+self.p-r+1]-self.u[i])
                temp.append((1-a_ir)*P[i-dis-1]+a_ir*P[i-dis])
            P[k-self.p+r-dis:k-sk+1-dis] = np.array(temp)
        # the last value is what we want
        return P[-1]


    def bs(self,us):
        y = []
        for x in us:
            y.append(self.De_Boor(x))
        y = np.array(y)
        return y


    def estimate_parameters(self,data_points,method="centripetal"):
        pts = data_points.copy()
        N = pts.shape[0]
        w = pts.shape[1]
        Li = []
        for i in range(1,N):
            Li.append(np.sum([pts[i,j] for j in range(w)])**0.5)
        L = np.sum(Li)

        t= [0]
        for i in range(len(Li)):
            Lki = 0
            for j in range(i+1):
                Lki += Li[j]
            t.append(Lki/L)
        t = np.array(t)
        self.paras = t
        ind = t>1.0
        t[ind] = 1.0
        return t


    def get_knots(self,method="average"):

        knots = np.zeros(self.p+1).tolist()

        paras_temp = self.paras.copy()
        # m = n+p+1
        self.m = self.n + self.p + 1
        # we only need m+1 knots
        # so we just select m+1-(p+1)-(p+1)+(p-1)+1+1  paras to average
        num = self.m - self.p  # select n+1 paras

        ind = np.linspace(0,paras_temp.shape[0]-1,num)
        ind = ind.astype(int)
        paras_knots = paras_temp[ind]

        for j in range(1,self.n-self.p+1):
            k_temp = 0
            # the maximun of variable i is n-1
            for i in range(j,j+self.p-1+1):
                k_temp += paras_knots[i]
            k_temp /= self.p
            knots.append(k_temp)

        add = np.ones(self.p+1).tolist()
        knots = knots + add
        knots = np.array(knots)
        self.u = knots
        self.m = knots.shape[0]-1
        return knots


    def set_paras(self,parameters):
        self.paras = parameters


    def set_knots(self,knots):
        self.u = knots


    def approximation(self,pts):
        ## Obtain a set of parameters t0, ..., tn
        #pts_paras = self.estimate_parameters(pts)
        ## knot vector U;
        #knots = self.get_knots()
        num = pts.shape[0]-1 # (num+1) is the number of data points

        P = np.zeros((self.n+1,pts.shape[1]),dtype=np.float64) # n+1 control points
        P[0] = pts[0]
        P[-1] = pts[-1]

        # compute N
        N = []
        for uq in self.paras:
            N_temp = self.coeffs(uq)
            N.append(N_temp)
        N = np.array(N)

        Q = [0] # hold the location
        for k in range(1,num-1+1):
            Q_temp = pts[k] - N[k,0]*pts[0] - N[k,self.n]*pts[-1]
            Q.append(Q_temp)

        b = [0]
        for i in range(1,self.n-1+1):
            b_temp = 0
            for k in range(1,num-1+1):
                b_temp += N[k,i]*Q[k]
            b.append(b_temp)

        b = b[1::]
        b = np.array(b)

        N = N[:,1:(self.n-1)+1]
        A = np.dot(N.T,N)
        cpm = np.linalg.solve(A,b)
        P[1:self.n] = cpm
        self.cp = P
        return P


if __name__ =="__main__":
    bs = BS_curve(8,3)
    #xx = np.linspace(0,4*np.pi,101)
    xx = np.random.normal(0, 1, 101)
    xx = np.linspace(0,max(xx),101)
    yy = np.exp(xx)/(np.exp(xx)+1)+np.random.normal(0, 1, 101)
    yy = np.sin(xx)
    #fig = plt.figure(figsize=(10,5))
    #ax = fig.add_subplot(111)
    #ax.scatter(xx,yy)

    data = np.array([xx,yy]).T
    paras = bs.estimate_parameters(data)
    knots = bs.get_knots()
    if bs.check():
        cp = bs.approximation(data)

    uq = np.linspace(0,1,101)
    y = bs.bs(uq)


In [16]:
cp

array([[0.        , 0.        ],
       [0.34964303, 0.34989951],
       [0.54721215, 0.52773269],
       [0.90168375, 0.79857378],
       [1.16127966, 0.92574977],
       [1.44932528, 1.00848988],
       [1.82992218, 0.99645041],
       [2.11181722, 0.87310372],
       [2.29880685, 0.74649965]])

In [17]:

def jacobi(A, b, x0, tol, n_iterations=300):
    """
    Performs Jacobi iterations to solve the line system of
    equations, Ax=b, starting from an initial guess, ``x0``.
    
    Returns:
    x, the estimated solution
    """
    
    n = A.shape[0]
    x = x0.copy()
    x_prev = x0.copy()
    counter = 0
    x_diff = tol+1
    
    while (x_diff > tol) and (counter < n_iterations): #iteration level
        for i in range(0, n): #element wise level for x
            s = 0
            for j in range(0,n): #summation for i !=j
                if i != j:
                    s += A[i,j] * x_prev[j] 
            
            x[i] = (b[i] - s) / A[i,i]
        #update values
        counter += 1
        x_diff = (np.sum((x-x_prev)**2))**0.5 
        x_prev = x.copy() #use new x for next iteration
        
    
    print("Number of Iterations: ", counter)
    print("Norm of Difference: ", x_diff)
    return x

In [18]:
def natural_cubic_spline(x, y, tol = 1e-100):
    """
    Interpolate using natural cubic splines.
    
    Generates a strictly diagonal dominant matrix then applies Jacobi's method.
    
    Returns coefficients:
    b, coefficient of x of degree 1
    c, coefficient of x of degree 2
    d, coefficient of x of degree 3
    """ 
    x = np.array(x)
    y = np.array(y)
    ### check if sorted
    if np.any(np.diff(x) < 0):
        idx = np.argsort(x)
        x = x[idx]
        y = y[idx]

    size = len(x)
    delta_x = np.diff(x)
    delta_y = np.diff(y)
    
    ### Get matrix A
    A = np.zeros(shape = (size,size))
    b = np.zeros(shape=(size,1))
    A[0,0] = 1
    A[-1,-1] = 1
    
    for i in range(1,size-1):
        A[i, i-1] = delta_x[i-1]
        A[i, i+1] = delta_x[i]
        A[i,i] = 2*(delta_x[i-1]+delta_x[i])
    ### Get matrix b
        b[i,0] = 3*(delta_y[i]/delta_x[i] - delta_y[i-1]/delta_x[i-1])
        
    ### Solves for c in Ac = b
    print('Jacobi Method Output:')
    c = jacobi(A, b, np.zeros(len(A)), tol = tol, n_iterations=1000)
    
    ### Solves for d and b
    d = np.zeros(shape = (size-1,1))
    b = np.zeros(shape = (size-1,1))
    for i in range(0,len(d)):
        d[i] = (c[i+1] - c[i]) / (3*delta_x[i])
        b[i] = (delta_y[i]/delta_x[i]) - (delta_x[i]/3)*(2*c[i] + c[i+1])    
    
    return b.squeeze(), c.squeeze(), d.squeeze()

In [15]:
 x = np.random.normal(0, 1, 100)
 y = np.exp(x)/( np.exp(x)+1)+np.random.normal(0, 1, 100)
 natural_cubic_spline(x, y, tol = 1e-100)

Jacobi Method Output:
Number of Iterations:  1000
Norm of Difference:  4.067383956680332e-12


(array([-9.94522306e+00, -2.39362250e+00,  8.32281477e+00,  1.51295362e-01,
        -4.80496722e+01, -7.12070119e+01, -5.95791067e+01,  7.64176851e+01,
         3.88219591e+00, -4.82011391e+01, -3.68162154e+01, -5.91874504e+01,
         1.17552040e+02,  2.04246255e+02, -5.07133268e+02, -4.90261715e+02,
         1.23268697e+02,  8.28605591e+01,  8.71140446e+01, -2.83946202e+01,
        -9.50842514e+01, -3.48284033e+02,  2.24814455e+03, -5.77547183e+03,
        -6.01805516e+03,  7.09847832e+02,  1.28949651e+02,  9.23434878e+01,
        -6.78827158e+01, -1.23174158e+02,  8.07171211e+00, -6.64847074e+02,
        -6.59300246e+02,  1.63798035e+02,  1.75673614e+02,  2.17617119e+02,
        -3.37754663e+02,  4.25611898e+02,  4.85109351e+02, -2.57522865e+02,
        -1.14876726e+01,  1.18468928e+02,  9.59713681e+00, -1.59365114e+02,
         1.58914279e+02,  1.04640296e+02,  2.15055301e+02,  2.58283314e+02,
        -3.99027732e+02,  7.16215570e+02,  1.39884895e+03, -2.08581590e+02,
         4.2